<a href="https://colab.research.google.com/github/axelpuyo/liasd/blob/master/explainers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
def rise(model, image, label, num_masks, proba):
  old_loss = model.evaluate(image[np.newaxis], label[np.newaxis], verbose=0)[0] # batch size = 1
  if not old_loss:
    old_loss += 1e-15

  masks = np.zeros((num_masks, image.shape[0], image.shape[1]))
  counts = np.zeros(masks.shape)
  new_image = np.zeros(image.shape)
  for n in range(num_masks):
    masks[n] = np.kron(np.random.choice([0, 1], size=(int(np.floor(image.shape[0]/2)), int(np.floor(image.shape[1]/2))), p=[proba, 1 - proba]), np.ones((2,2)))
    for k in range(3):
      new_image[..., k] = image[..., k]*masks[n]

    new_loss = model.evaluate(new_image[np.newaxis], label[np.newaxis], verbose=0)[0]
    indexes = (masks[n] == 0)
    counts[n, indexes == True] = new_loss
  
  heatmap = np.average(counts, axis=0)
  
  return heatmap


In [3]:
def gradcam(model, layer_name, img, pred_index=None):
  ## Create separate models for the convolutional block (feature extraction) and the dense layer block (classification)
  # Feature extraction model
  print(layer_name)
  grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )

  ## Gradient extraction
  with tf.GradientTape() as tape:
        features, preds = grad_model(img)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

  # Last gradient extraction
  grads = tape.gradient(class_channel, features)
  pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

  # Multiply features by "how important" it is wrt. the gradient
  features = features.numpy()[0]
  pooled_grads = pooled_grads.numpy()

  heatmap = tf.squeeze(features @ pooled_grads[..., tf.newaxis])
  heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

  return heatmap


In [4]:
def shap():
  pass